In [33]:
from pyincore import IncoreClient, Dataset
from pyincore_data.censusutil import CensusUtil
from pyincore.analyses.housingrecoverysequential import HousingRecoverySequential
from pyincore.analyses.socialvulnerability import SocialVulnerability

import pandas as pd

In [34]:
# Connect to IN-CORE service
client = IncoreClient()

Connection successful to IN-CORE services. pyIncore version detected: 1.8.0


### Chaining with social vulnerability analysis

#### prepare input datasets for social vulnerability analysis

In [35]:
# Parameters
state = "texas"
county = "galveston"
year = 2020

In [36]:
# get fips code to use fetch census data
fips = CensusUtil.get_fips_by_state_county(state=state, county=county)
state_code = fips[:2]
county_code = fips[2:]
fips

'48167'

In [37]:
navs = CensusUtil.national_ave_values(year=year)
national_vulnerability_feature_averages = Dataset.from_csv_data(navs, name="national_vulnerability_feature_averages.csv",
                                                                 data_type="incore:socialVulnerabilityFeatureAverages")

geo_type = "block%20group:*"
# geo_type = "tract:*"
social_vulnerability_dem_factors_df = CensusUtil.demographic_factors(state_code, county_code, year=year,
                                                                      geo_type=geo_type)

# Temp fix: remove bad data point
social_vulnerability_dem_factors_df = social_vulnerability_dem_factors_df.dropna()

social_vulnerability_dem_factors = Dataset.from_dataframe(social_vulnerability_dem_factors_df,
                                                           name="social_vunlnerability_dem_factors",
                                                           data_type="incore:socialVulnerabilityDemFactors")

csv saved as api_20230217-141539.csv
csv saved as api_20230217-141540.csv
csv saved as api_20230217-141540.csv
csv saved as api_20230217-141541.csv
csv saved as api_20230217-141542.csv
csv saved as api_20230217-141543.csv


/Users/mohanar2/opt/anaconda3/envs/pyincoreEnv/lib/python3.8/site-packages/pyincore_data/censusutil.py:354: RuntimeWarning: invalid value encountered in long_scalars
  lambda row: row["B03002_003E"] / row["B03002_001E"], axis=1)


csv saved as api_20230217-141543.csv


/Users/mohanar2/opt/anaconda3/envs/pyincoreEnv/lib/python3.8/site-packages/pyincore_data/censusutil.py:360: RuntimeWarning: invalid value encountered in long_scalars
  df_2["factor_owner_occupied"] = df_2.astype(int).apply(lambda row: row["B25003_002E"] / row["B25003_001E"],


csv saved as api_20230217-141544.csv


/Users/mohanar2/opt/anaconda3/envs/pyincoreEnv/lib/python3.8/site-packages/pyincore_data/censusutil.py:370: RuntimeWarning: invalid value encountered in long_scalars
  lambda row: 1 - row["B17021_002E"] / row["B17021_001E"], axis=1)


csv saved as api_20230217-141545.csv


/Users/mohanar2/opt/anaconda3/envs/pyincoreEnv/lib/python3.8/site-packages/pyincore_data/censusutil.py:380: RuntimeWarning: invalid value encountered in long_scalars
  lambda row: (row["B15003_017E"]


csv saved as api_20230217-141546.csv


/Users/mohanar2/opt/anaconda3/envs/pyincoreEnv/lib/python3.8/site-packages/pyincore_data/censusutil.py:410: RuntimeWarning: divide by zero encountered in long_scalars
  df_5.astype(int).apply(lambda row: (row['C21007_006E']
/Users/mohanar2/opt/anaconda3/envs/pyincoreEnv/lib/python3.8/site-packages/pyincore_data/censusutil.py:410: RuntimeWarning: invalid value encountered in long_scalars
  df_5.astype(int).apply(lambda row: (row['C21007_006E']


#### run social vulnerability analysis

In [38]:
# Configure pyincore access and the analysis with the information above

client = IncoreClient()
social_vulnerability = SocialVulnerability(client)

social_vulnerability.set_parameter("result_name", "social_vulnerabilty")
social_vulnerability.set_input_dataset("national_vulnerability_feature_averages", national_vulnerability_feature_averages)
social_vulnerability.set_input_dataset("social_vulnerability_demographic_factors", social_vulnerability_dem_factors)

Connection successful to IN-CORE services. pyIncore version detected: 1.8.0


True

In [39]:
# Run social vulnerability damage analysis
result = social_vulnerability.run_analysis()

In [40]:
# Retrieve result dataset
sv_result = social_vulnerability.get_output_dataset("sv_result")

# Convert dataset to Pandas DataFrame
df = sv_result.get_dataframe_from_csv()

# Display top 5 rows of output data
df.head()

,GEO_ID,factor_white_nonHispanic,factor_owner_occupied,factor_earning_higher_than_national_poverty_rate,factor_over_25_with_high_school_diploma_or_higher,factor_without_disability_age_18_to_65,FIPS,R1,R2,R3,R4,R5,SVS,zone
0,1500000US481677201001,0.826873,0.857143,0.836348,0.985014,0.075404,481677201001,1.389439,1.330196,0.962965,1.114087,0.137041,0.986746,Medium Vulnerable (zone3)
1,1500000US481677201002,0.676741,0.889262,0.894875,0.929273,0.176230,481677201002,1.137164,1.380041,1.030353,1.051042,0.320285,0.983777,Medium Vulnerable (zone3)
2,1500000US481677201003,0.517323,0.809955,0.984252,0.980989,0.056364,481677201003,0.869285,1.256965,1.133261,1.109534,0.102437,0.894296,Medium to High Vulnerable (zone4)
3,1500000US481677201004,0.561679,1.000000,1.000000,0.919908,0.061051,481677201004,0.943819,1.551895,1.151393,1.040451,0.110956,0.959703,Medium Vulnerable (zone3)
4,1500000US481677202001,0.701548,0.973113,0.987214,0.973601,0.062718,481677202001,1.178848,1.510169,1.136671,1.101179,0.113985,1.008170,Medium Vulnerable (zone3)


### Household-level housing serial recovery for Galveston testbed.

In [41]:
# Population dislocation result aggregated to the block group level.
population_dislocation = "60f5ea5d544e944c3cec66dd" # Galveston testbed
# Transition probability matrix per social vulnerability level.
transition_probability_matrix = "60f5e2ae544e944c3cec0794"
# Initial mass probability function for stage 0.
initial_probability_vector = "60f5e918544e944c3cec668b"

In [42]:
# Create housing recovery instance
housing_recovery = HousingRecoverySequential(client)

# Load input datasets
housing_recovery.load_remote_input_dataset("population_dislocation_block", population_dislocation)
housing_recovery.load_remote_input_dataset("tpm", transition_probability_matrix)
housing_recovery.load_remote_input_dataset("initial_stage_probabilities", initial_probability_vector)

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


In [43]:
# Chain with SV output
housing_recovery.set_input_dataset('sv_result', sv_result)

True

In [44]:
# Initial value to seed the random number generator to ensure replication
seed = 1234
# A size of the analysis time step.
t_delta = 1.0
# Total duration.
t_final = 90.0

In [45]:
# Specify the result name
result_name = "housing_recovery_result"

# Set analysis parameters
housing_recovery.set_parameter("result_name", result_name)
housing_recovery.set_parameter("seed", seed)
housing_recovery.set_parameter("t_delta", t_delta)
housing_recovery.set_parameter("t_final", t_final)

True

In [ ]:
# Run the analysis
housing_recovery.run()

In [ ]:
# Retrieve result dataset
result = housing_recovery.get_output_dataset("ds_result")

# Convert dataset to Pandas DataFrame
df = result.get_dataframe_from_csv()

# Display top 5 rows of output data
df.head()